In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import zipfile
with zipfile.ZipFile('/content/drive/My Drive/Research/Kaggle/Covid-19/CORD-19-research-challenge.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/CORD-19-research-challenge')

# Load data

In [0]:
import numpy as np
import pandas as pd
import glob
import json

import matplotlib.pyplot as plt
plt.style.use('ggplot')

## Metadata

In [5]:
metadata_path = f'/content/CORD-19-research-challenge/metadata.csv'
meta_df = pd.read_csv(metadata_path, sep=',')
meta_df

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_pdf_parse,has_pmc_xml_parse,full_text_file,url
0,xqhn0vbp,1e1286db212100993d03cc22374b624f7caee956,PMC,Airborne rhinovirus detection and effect of ul...,10.1186/1471-2458-3-5,PMC140314,12525263.0,no-cc,"BACKGROUND: Rhinovirus, the most common cause ...",2003-01-13,"Myatt, Theodore A; Johnston, Sebastian L; Rudn...",BMC Public Health,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...
1,gi6uaa83,8ae137c8da1607b3a8e4c946c07ca8bda67f88ac,PMC,Discovering human history from stomach bacteria,10.1186/gb-2003-4-5-213,PMC156578,12734001.0,no-cc,Recent analyses of human pathogens have reveal...,2003-04-28,"Disotell, Todd R",Genome Biol,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...
2,le0ogx1s,NaN,PMC,A new recruit for the army of the men of death,10.1186/gb-2003-4-7-113,PMC193621,12844350.0,no-cc,"The army of the men of death, in John Bunyan's...",2003-06-27,"Petsko, Gregory A",Genome Biol,NaN,NaN,False,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...
3,fy4w7xz8,0104f6ceccf92ae8567a0102f89cbb976969a774,PMC,Association of HLA class I with severe acute r...,10.1186/1471-2350-4-9,PMC212558,12969506.0,no-cc,BACKGROUND: The human leukocyte antigen (HLA) ...,2003-09-12,"Lin, Marie; Tseng, Hsiang-Kuang; Trejaut, Jean...",BMC Med Genet,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...
4,0qaoam29,5b68a553a7cbbea13472721cd1ad617d42b40c26,PMC,A double epidemic model for the SARS propagation,10.1186/1471-2334-3-19,PMC222908,12964944.0,no-cc,BACKGROUND: An epidemic of a Severe Acute Resp...,2003-09-10,"Ng, Tuen Wai; Turinici, Gabriel; Danchin, Antoine",BMC Infect Dis,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52393,g1ie0ahz,7b94d744920dd9f5c6ea3e53b50755919f3fc379,Elsevier,Characterization and inhibition of norovirus p...,10.1016/j.virol.2011.12.002,PMC3259199,22200497.0,els-covid,Abstract Noroviruses are the major cause of fo...,2012-02-20,"Chang, Kyeong-Ok; Takahashi, Daisuke; Prakash,...",Virology,NaN,NaN,True,False,custom_license,https://doi.org/10.1016/j.virol.2011.12.002
52394,4pztbu51,26273a8abe98bbeabfe8651fc433188c5f5b82c4; d33b...,Elsevier,A Heuristic Indication and Warning Staging Mod...,10.1197/jamia.m2558,PMC2274782,18096906.0,els-covid,Abstract Objective This paper presents a model...,2008-04-30,"Wilson, James M.; Polyak, Marat G.; Blake, Jan...",Journal of the American Medical Informatics As...,NaN,NaN,True,False,custom_license,https://doi.org/10.1197/jamia.m2558
52395,h10gvuec,8fa2b64359e82ce74886b99441e04210810e81a5,Elsevier,Update on SARS research and other possibly zoo...,10.1016/j.ijantimicag.2010.06.016,PMC7126933,20801001.0,els-covid,Abstract The global outbreak of severe acute r...,2010-11-30,"Anderson, Larry J.; Tong, Suxiang",International Journal of Antimicrobial Agents,NaN,NaN,True,False,custom_license,https://doi.org/10.1016/j.ijantimicag.2010.06.016
52396,xo4mbxei,9d366e500df1adf1976c8d8acecc7a77c1510282,Elsevier,De novo generation of defective interfering RN...,10.1016/0042-6822(91)90484-s,PMC7131041,1994574.0,els-covid,Abstract Defective interfering (DI) RNAs were ...,1991-03-31,"Knorr, D.A.; Mullin, R.H.; Hearne, P.Q.; Morri...",Virology,NaN,NaN,True,False,custom_license,https://doi.org/10.1016/0042-6822(91)90484-s


In [6]:
meta_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52398 entries, 0 to 52397
Data columns (total 18 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   cord_uid                     52398 non-null  object 
 1   sha                          39024 non-null  object 
 2   source_x                     52398 non-null  object 
 3   title                        52240 non-null  object 
 4   doi                          49058 non-null  object 
 5   pmcid                        43652 non-null  object 
 6   pubmed_id                    38058 non-null  float64
 7   license                      52398 non-null  object 
 8   abstract                     43168 non-null  object 
 9   publish_time                 52390 non-null  object 
 10  authors                      50119 non-null  object 
 11  journal                      47156 non-null  object 
 12  Microsoft Academic Paper ID  964 non-null    float64
 13  WHO #Covidence  

# Helper Functions

In [0]:
def format_name(author):
    middle_name = " ".join(author['middle'])
    
    if author['middle']:
        return " ".join([author['first'], middle_name, author['last']])
    else:
        return " ".join([author['first'], author['last']])

def format_affiliation(affiliation):
    text = []
    location = affiliation.get('location')
    if location:
        text.extend(list(affiliation['location'].values()))
    
    institution = affiliation.get('institution')
    if institution:
        text = [institution] + text
    return ", ".join(text)

def format_authors(authors, with_affiliation=False):
    name_ls = []
    
    for author in authors:
        name = format_name(author)
        if with_affiliation:
            affiliation = format_affiliation(author['affiliation'])
            if affiliation:
                name_ls.append(f"{name} ({affiliation})")
            else:
                name_ls.append(name)
        else:
            name_ls.append(name)
    
    return ", ".join(name_ls)

def format_body(body_text):
    texts = [(di['section'], di['text']) for di in body_text]
    texts_di = {di['section']: "" for di in body_text}
    
    for section, text in texts:
        texts_di[section] += text

    body = ""

    for section, text in texts_di.items():
        body += section
        body += "\n\n"
        body += text
        body += "\n\n"
    
    return body

def format_bib(bibs):
    if type(bibs) == dict:
        bibs = list(bibs.values())
    bibs = deepcopy(bibs)
    formatted = []
    
    for bib in bibs:
        bib['authors'] = format_authors(
            bib['authors'], 
            with_affiliation=False
        )
        formatted_ls = [str(bib[k]) for k in ['title', 'authors', 'venue', 'year']]
        formatted.append(", ".join(formatted_ls))

    return "; ".join(formatted)

In [0]:
def load_files(dirname):
    filenames = os.listdir(dirname)
    raw_files = []

    for filename in tqdm(filenames):
        filename = dirname + filename
        file = json.load(open(filename, 'rb'))
        raw_files.append(file)
    
    return raw_files

def generate_clean_df(all_files):
    cleaned_files = []
    
    for file in tqdm(all_files):
        features = [
            file['paper_id'],
            file['metadata']['title'],
            format_authors(file['metadata']['authors']),
            format_authors(file['metadata']['authors'], 
                           with_affiliation=True),
            format_body(file['abstract']),
            format_body(file['body_text']),
            format_bib(file['bib_entries']),
            file['metadata']['authors'],
            file['bib_entries']
        ]

        cleaned_files.append(features)

    col_names = ['paper_id', 'title', 'authors',
                 'affiliations', 'abstract', 'text', 
                 'bibliography','raw_authors','raw_bibliography']

    clean_df = pd.DataFrame(cleaned_files, columns=col_names)
    clean_df.head()
    
    return clean_df

# Biorxiv

In [0]:
import os
from pprint import pprint
from copy import deepcopy
from tqdm.notebook import tqdm

In [10]:
#comm_dir = '/content/CORD-19-research-challenge/comm_use_subset/comm_use_subset/pdf_json/'
biorxiv_dir = '/content/CORD-19-research-challenge/biorxiv_medrxiv/biorxiv_medrxiv/pdf_json/'
filenames = os.listdir(biorxiv_dir)
print("Number of articles retrieved from comm_dir:", len(filenames))

Number of articles retrieved from comm_dir: 1934


In [0]:
all_files = []

for filename in filenames:
    filename = biorxiv_dir + filename

    try:
      file = json.load(open(filename, 'rb'))
    except ValueError as err:
      print(err)

    all_files.append(file)

In [12]:
file = all_files[0]
print("Dictionary keys:", file.keys())

Dictionary keys: dict_keys(['paper_id', 'metadata', 'abstract', 'body_text', 'bib_entries', 'ref_entries', 'back_matter'])


In [13]:
# Biorxiv: Abstract
pprint(file['abstract'])

[{'cite_spans': [],
  'ref_spans': [],
  'section': 'Abstract',
  'text': 'Background: The world has been put in an unprecedented situation by '
          'the Covid-19 pandemic. Creating models to describe and quantify '
          'alternative mitigation strategies becomes increasingly urgent.'},
 {'cite_spans': [],
  'ref_spans': [],
  'section': 'Abstract',
  'text': 'Methods: In this study, we propose an agent-based model of disease '
          'transmission in a society divided into closely connected families, '
          'workplaces, and social groups. This allows us to discuss mitigation '
          'strategies, including targeted quarantine measures.'},
 {'cite_spans': [],
  'ref_spans': [],
  'section': 'Abstract',
  'text': 'Results: We find that workplace and more diffuse social contacts '
          'are roughly equally important to disease spread, and that an '
          'effective lockdown must target both. We examine the cost-benefit of '
          'replacing a lockdown w

In [14]:
# Biorxiv: body text
print("body_text type:", type(file['body_text']))
print("body_text length:", len(file['body_text']))
print("body_text keys:", file['body_text'][0].keys())

body_text type: <class 'list'>
body_text length: 39
body_text keys: dict_keys(['text', 'cite_spans', 'ref_spans', 'section'])


In [15]:
print("body_text content:")
pprint(file['body_text'][:2], depth=3)

body_text content:
[{'cite_spans': [{...}],
  'ref_spans': [],
  'section': 'Introduction',
  'text': 'The 2020 coronavirus (Covid-19) pandemic has lead to authorities '
          'worldwide requiring guidance as to what mitigation efforts will '
          'work in controlling disease outbreaks. In this endeavor, '
          'theoretical modelling can play a crucial role. Traditional '
          'epidemiological models that assume universal or constant infection '
          'parameters are not sufficient to address case specific strategies '
          'like contact tracing. Therefore, we have developed an agent-based '
          'epidemiological model which takes into account the fact that '
          'disease transmission is differentiated in distinct arenas of social '
          'life that each play a different role in a lockdown: The family, the '
          'workplace, our social circles, and the public sphere. This '
          'subdivision becomes especially important when discussi

In [16]:
texts = [(di['section'], di['text']) for di in file['body_text']]
texts_di = {di['section']: "" for di in file['body_text']}
for section, text in texts:
    texts_di[section] += text

pprint(list(texts_di.keys()))

['Introduction',
 'Methods',
 'Results: Mitigation strategies',
 '4/8',
 '5/8',
 'Discussion',
 '6/8']


In [17]:
body = ""

for section, text in texts_di.items():
    body += section
    body += "\n\n"
    body += text
    body += "\n\n"

print(body[:3000])

Introduction

The 2020 coronavirus (Covid-19) pandemic has lead to authorities worldwide requiring guidance as to what mitigation efforts will work in controlling disease outbreaks. In this endeavor, theoretical modelling can play a crucial role. Traditional epidemiological models that assume universal or constant infection parameters are not sufficient to address case specific strategies like contact tracing. Therefore, we have developed an agent-based epidemiological model which takes into account the fact that disease transmission is differentiated in distinct arenas of social life that each play a different role in a lockdown: The family, the workplace, our social circles, and the public sphere. This subdivision becomes especially important when discussing such efforts as contact tracing. Using an estimated weight of social contacts within each of these four spheres 1 we discuss effect of various mitigation strategies.At the time of writing, both classical mean field models 2 and a

In [18]:
print(format_body(file['body_text'])[:3000])

Introduction

The 2020 coronavirus (Covid-19) pandemic has lead to authorities worldwide requiring guidance as to what mitigation efforts will work in controlling disease outbreaks. In this endeavor, theoretical modelling can play a crucial role. Traditional epidemiological models that assume universal or constant infection parameters are not sufficient to address case specific strategies like contact tracing. Therefore, we have developed an agent-based epidemiological model which takes into account the fact that disease transmission is differentiated in distinct arenas of social life that each play a different role in a lockdown: The family, the workplace, our social circles, and the public sphere. This subdivision becomes especially important when discussing such efforts as contact tracing. Using an estimated weight of social contacts within each of these four spheres 1 we discuss effect of various mitigation strategies.At the time of writing, both classical mean field models 2 and a

In [19]:
# Biorxiv: Metadata
print(all_files[0]['metadata'].keys())

dict_keys(['title', 'authors'])


In [20]:
print(all_files[0]['metadata']['title'])

Estimating cost-benefit of quarantine length for Covid-19 mitigation


In [21]:
authors = all_files[0]['metadata']['authors']
pprint(authors[:3])

[{'affiliation': {'institution': 'Niels Bohr Institute',
                  'laboratory': '',
                  'location': {'country': 'Denmark',
                               'postCode': '2100',
                               'settlement': 'København Ø'}},
  'email': '',
  'first': 'Andreas',
  'last': 'Eilersen',
  'middle': [],
  'suffix': ''},
 {'affiliation': {'institution': 'Niels Bohr Institute',
                  'laboratory': '',
                  'location': {'country': 'Denmark',
                               'postCode': '2100',
                               'settlement': 'København Ø'}},
  'email': '',
  'first': '+',
  'last': '',
  'middle': [],
  'suffix': ''},
 {'affiliation': {'institution': 'Niels Bohr Institute',
                  'laboratory': '',
                  'location': {'country': 'Denmark',
                               'postCode': '2100',
                               'settlement': 'København Ø'}},
  'email': '*ksneppen@gmail.com',
  'first': 'Kim',
 

In [22]:
for author in authors:
    print("Name:", format_name(author))
    print("Affiliation:", format_affiliation(author['affiliation']))
    print()

Name: Andreas Eilersen
Affiliation: Niels Bohr Institute, 2100, København Ø, Denmark

Name: + 
Affiliation: Niels Bohr Institute, 2100, København Ø, Denmark

Name: Kim Sneppen
Affiliation: Niels Bohr Institute, 2100, København Ø, Denmark



In [22]:
pprint(all_files[4]['metadata'], depth=4)

{'authors': [{'affiliation': {'institution': 'Sichuan University West China '
                                             'Hospital',
                              'laboratory': '',
                              'location': {...}},
              'email': '',
              'first': 'Cheng',
              'last': 'Long',
              'middle': [],
              'suffix': ''},
             {'affiliation': {'institution': 'The Chinese University of Hong '
                                             'Kong',
                              'laboratory': '',
                              'location': {...}},
              'email': '',
              'first': 'Tieyong',
              'last': 'Zeng',
              'middle': [],
              'suffix': ''},
             {'affiliation': {'institution': 'Fujian Normal University',
                              'laboratory': 'Provincial University Key '
                                            'Laboratory of Cellular Stress '
                    

In [23]:
authors = all_files[4]['metadata']['authors']
print("Formatting without affiliation:")
print(format_authors(authors, with_affiliation=False))
print("\nFormatting with affiliation:")
print(format_authors(authors, with_affiliation=True))

Formatting without affiliation:
Cheng Long, Tieyong Zeng, Xinmiao Fu

Formatting with affiliation:
Cheng Long (Sichuan University West China Hospital, Sichuan Province 8 610041, Chengdu City, China), Tieyong Zeng (The Chinese University of Hong Kong, 99999, Shatin, Hong Kong, NT, China), Xinmiao Fu (Fujian Normal University, 350117, Fuzhou City, China)


In [24]:
# Biorxiv: bibliography
bibs = list(file['bib_entries'].values())
pprint(bibs[:2], depth=4)

[{'authors': [{'first': 'J', 'last': 'Mossong', 'middle': [], 'suffix': ''}],
  'issn': '',
  'other_ids': {},
  'pages': '',
  'ref_id': 'b0',
  'title': 'Social contacts and mixing patterns relevant to the spread of '
           'infectious diseases',
  'venue': 'PLoS medicine',
  'volume': '5',
  'year': 2008},
 {'authors': [{'first': 'L', 'last': 'Peng', 'middle': [], 'suffix': ''},
              {'first': 'W', 'last': 'Yang', 'middle': [], 'suffix': ''},
              {'first': 'D', 'last': 'Zhang', 'middle': [], 'suffix': ''},
              {'first': 'C', 'last': 'Zhuge', 'middle': [], 'suffix': ''},
              {'first': 'L', 'last': 'Hong', 'middle': [], 'suffix': ''}],
  'issn': '',
  'other_ids': {'arXiv': ['arXiv:2002.06563']},
  'pages': '',
  'ref_id': 'b1',
  'title': 'Epidemic analysis of covid-19 in china by dynamical modeling',
  'venue': '',
  'volume': '',
  'year': 2020}]


In [25]:
format_authors(bibs[1]['authors'], with_affiliation=False)

'L Peng, W Yang, D Zhang, C Zhuge, L Hong'

In [26]:
bib_formatted = format_bib(bibs[:5])
print(bib_formatted)

Social contacts and mixing patterns relevant to the spread of infectious diseases, J Mossong, PLoS medicine, 2008; Epidemic analysis of covid-19 in china by dynamical modeling, L Peng, W Yang, D Zhang, C Zhuge, L Hong, , 2020; Modelling transmission and control of the covid-19 pandemic in australia, S L Chang, N Harding, C Zachreson, O M Cliff, M Prokopenko, , 2020; Contagion! the bbc four pandemic-the model behind the documentary, P Klepac, S Kissler, J Gog, Epidemics, 2018; Contacts in context: large-scale setting-specific social mixing matrices from the bbc pandemic project, P Klepac, , 2020


In [27]:
# Biorxiv: Generate CSV
cleaned_files = []

for file in tqdm(all_files):
    features = [
        file['paper_id'],
        file['metadata']['title'],
        format_authors(file['metadata']['authors']),
        format_authors(file['metadata']['authors'], 
                       with_affiliation=True),
        format_body(file['abstract']),
        format_body(file['body_text']),
        format_bib(file['bib_entries']),
        file['metadata']['authors'],
        file['bib_entries']
    ]
    
    cleaned_files.append(features)

In [28]:
col_names = [
    'paper_id', 
    'title', 
    'authors',
    'affiliations', 
    'abstract', 
    'text', 
    'bibliography',
    'raw_authors',
    'raw_bibliography'
]

clean_df = pd.DataFrame(cleaned_files, columns=col_names)
clean_df.head()

,paper_id,title,authors,affiliations,abstract,text,bibliography,raw_authors,raw_bibliography
0,455a70ee73cd1eb54682d2b3dcd3754eec191816,Estimating cost-benefit of quarantine length f...,"Andreas Eilersen, + , Kim Sneppen","Andreas Eilersen (Niels Bohr Institute, 2100, ...",Abstract\n\nBackground: The world has been put...,Introduction\n\nThe 2020 coronavirus (Covid-19...,Social contacts and mixing patterns relevant t...,"[{'first': 'Andreas', 'middle': [], 'last': 'E...","{'BIBREF0': {'ref_id': 'b0', 'title': 'Social ..."
1,8a407d05ad59cc54944a5e2d56dd0decd3572342,Air Pollution Reduction and Mortality Benefit ...,"Kai Chen, Meng Wang, Conghong Huang, Patrick L...","Kai Chen, Meng Wang (University at Buffalo Sch...",,LETTER: 14\n\nTo control the novel coronavirus...,Beijing-Tianjin-Hebei and surrounding areas an...,"[{'first': 'Kai', 'middle': [], 'last': 'Chen'...","{'BIBREF0': {'ref_id': 'b0', 'title': 'Beijing..."
2,0fd5170ad0080433dbe4ed11906969f93981e67d,Comparison of SARS-CoV-2 infections among 3 sp...,"Shuaiyao Lu 1#, Yuan Zhao, Wenhai Yu 1#, Yun Y...",Shuaiyao Lu 1# (Chinese Academy of Medical Sci...,"Abstract\n\nCOVID-19, caused by SARS-CoV-2 inf...",INTRODUCTION\n\nThe novel coronavirus disease ...,New Virus Discovered by Chinese Scientists Inv...,"[{'first': 'Shuaiyao', 'middle': ['Lu'], 'last...","{'BIBREF0': {'ref_id': 'b0', 'title': 'New Vir..."
3,4f8d24c531d2c334969e09e4b5aed66dcc925c4b,Association of radiologic findings with mortal...,"Mingli Yuan, Wen Yin, Zhaowu Tao, Weijun Tan, ...","Mingli Yuan, Wen Yin, Zhaowu Tao, Weijun Tan, ...",Abstract\n\nBackground: Radiologic characteris...,"Introduction\n\nIn December, 2019, a series of...",Wuhan municipal health commission's briefing o...,"[{'first': 'Mingli', 'middle': [], 'last': 'Yu...","{'BIBREF0': {'ref_id': 'b0', 'title': 'Wuhan m..."
4,bde55b7dd004dbe9406fb8289922dcaf27554164,Title Page Little Risk of the COVID-19 Resurge...,"Cheng Long, Tieyong Zeng, Xinmiao Fu",Cheng Long (Sichuan University West China Hosp...,Abstract\n\nObjective: School reopening has no...,\n\nmiddle and high schools in China (outside ...,National Health Commission of China: Update of...,"[{'first': 'Cheng', 'middle': [], 'last': 'Lon...","{'BIBREF0': {'ref_id': 'b0', 'title': 'Nationa..."


In [0]:
clean_df.to_csv('/content/drive/My Drive/Research/Kaggle/Covid-19/biorxiv_clean.csv', index=False)

In [30]:
# Generate CSV: Custom (PMC), Commercial, Non-commercial licenses
pmc_dir = '/content/CORD-19-research-challenge/custom_license/custom_license/pdf_json/'
pmc_files = load_files(pmc_dir)
pmc_df = generate_clean_df(pmc_files)
pmc_df.to_csv('/content/drive/My Drive/Research/Kaggle/Covid-19/clean_pmc.csv', index=False)
pmc_df.head()

,paper_id,title,authors,affiliations,abstract,text,bibliography,raw_authors,raw_bibliography
0,07b321dfe1534033413fa15aeadc1423f82747f6,,"Lia Van Der Hoek, Klaus Sure, Gabriele Ihorst,...","Lia Van Der Hoek, Klaus Sure, Gabriele Ihorst,...",,INTRODUCTION\n\nScreening of respiratory sampl...,"Identification of a new human coronavirus, L V...","[{'first': 'Lia', 'middle': [], 'last': 'Van D...","{'BIBREF0': {'ref_id': 'b0', 'title': 'Identif..."
1,541edacc94c9cf73a574164c975373aa4b366fc8,Coronavirus MHV-A59 infects the lung and cause...,"Zhangsheng Yang, Jun Du, Gang Chen, Jie Zhao, ...","Zhangsheng Yang (Chinese Academy of Sciences, ...","Abstract\n\nA-59 (MHV-A59), a hepatic and neur...","INTRODUCTION\n\nTwo novel betacoronaviruses, s...",Autoimmune hepatitis-like disease in C57BL/6 m...,"[{'first': 'Zhangsheng', 'middle': [], 'last':...","{'BIBREF0': {'ref_id': 'b0', 'title': 'Autoimm..."
2,d4c584e9784624305337e74b2918e162594673df,,,,,\n\nThis chapter covers the pharmacology of ch...,Disease modifying drugs for progressive rheuma...,[],"{'BIBREF0': {'ref_id': 'b0', 'title': 'Disease..."
3,3f86a6ff036585c0f052e75b9f183629c0b6aa96,Regulations Regulatory oversight of human path...,"Labrie C , Lecordier","Labrie C (Public Health Agency of Canada, Ott...","Abstract\n\nFrom 1994 to 2009, federal oversig...","Introduction\n\nSince 2001, there have been nu...",Famous cases and criminals: Amerithrax or Anth...,"[{'first': 'Labrie', 'middle': ['C'], 'last': ...","{'BIBREF0': {'ref_id': 'b0', 'title': 'Famous ..."
4,b0bbd7b0db45e8b9a817d7f403f1e705821943d3,WhyZombie? Zombie Pop Culture to Improve Infec...,"Peta-Anne Zimmerman, Matt Mason","Peta-Anne Zimmerman, Matt Mason",,\n\nPopular culture and urban mythology are kn...,"History of Zombies, , , 2016; Ontological Anxi...","[{'first': 'Peta-Anne', 'middle': [], 'last': ...","{'BIBREF0': {'ref_id': 'b0', 'title': 'History..."


In [23]:
comm_dir = '/content/CORD-19-research-challenge/comm_use_subset/comm_use_subset/pdf_json/'
comm_files = load_files(comm_dir)
comm_df = generate_clean_df(comm_files)
comm_df.to_csv('/content/drive/My Drive/Research/Kaggle/Covid-19/clean_comm_use.csv', index=False)
comm_df.head()

,paper_id,title,authors,affiliations,abstract,text,bibliography,raw_authors,raw_bibliography
0,d336ffee34085032c158363c77b69204c58ca85a,Type III IFN Receptor Expression and Functiona...,"Peng Zhou, Chris Cowled, Glenn A Marsh, Zhengl...","Peng Zhou (CSIRO Livestock Industries, Geelong...",Abstract\n\nBats are rich reservoir hosts for ...,Introduction\n\nThe hallmark of the innate imm...,IFN-lambdas mediate antiviral protection throu...,"[{'first': 'Peng', 'middle': [], 'last': 'Zhou...","{'BIBREF0': {'ref_id': 'b0', 'title': 'IFN-lam..."
1,144bd370f94a0844a954100eb08bf1df9403ddaf,Bacterial ribonuclease binase exerts an intra-...,"Raihan Shah, Mahmud , Ahmed Mostafa, Christin ...","Raihan Shah, Mahmud , Ahmed Mostafa, Christin ...",Abstract\n\nBackground: Influenza is a severe ...,Background\n\nInfluenza A virus (IAV) is an RN...,"Seasonal influenza and vaccine herd effect, T ...","[{'first': 'Raihan', 'middle': [], 'last': 'Sh...","{'BIBREF1': {'ref_id': 'b1', 'title': 'Seasona..."
2,b8a55c284d17093b3d9c033f5c74633f47aaabe3,"The IRIDICA BAC BSI Assay: Rapid, Sensitive an...","D Open Access Citation ; Metzgar, M W Frinder,...","D Open Access Citation ; Metzgar, M W Frinder,...",Abstract\n\nBloodstream infection (BSI) and se...,Introduction\n\nBloodstream infection (BSI) an...,PCR-based rapid sepsis diagnosis effectively g...,"[{'first': 'D', 'middle': [], 'last': 'Open Ac...","{'BIBREF0': {'ref_id': 'b0', 'title': 'PCR-bas..."
3,5206b0be1000e6c8d75a827af94c1b8e61e297c3,Travel Patterns in China,"T Garske, H Yu, Z Peng, M Ye, H Zhou","T Garske, H Yu, Z Peng, M Ye, H Zhou",Abstract\n\nThe spread of infectious disease e...,Introduction\n\nWorldwide urbanisation and inc...,A major outbreak of severe acute respiratory s...,"[{'first': 'T', 'middle': [], 'last': 'Garske'...","{'BIBREF0': {'ref_id': 'b0', 'title': 'A major..."
4,5a6e220c6796a02e36a272103dd39dd62c9311eb,Antibody Engineering for Pursuing a Healthier ...,"Afuh F U H Saeed, R Wang, Ling S Wang, S","Afuh F U H Saeed, R Wang, Ling S Wang, S",Abstract\n\nSince the development of antibody-...,"INTRODUCTION\n\nIn recent years, the developme...",Appearance of skin and nail toxicity in patien...,"[{'first': 'Afuh', 'middle': ['F U H'], 'last'...","{'BIBREF0': {'ref_id': 'b0', 'title': 'Appeara..."


In [24]:
noncomm_dir = '/content/CORD-19-research-challenge/noncomm_use_subset/noncomm_use_subset/pdf_json/'
noncomm_files = load_files(noncomm_dir)
noncomm_df = generate_clean_df(noncomm_files)
noncomm_df.to_csv('/content/drive/My Drive/Research/Kaggle/Covid-19/clean_noncomm_use.csv', index=False)
noncomm_df.head()

,paper_id,title,authors,affiliations,abstract,text,bibliography,raw_authors,raw_bibliography
0,8fefe8e70632ffdfd96f701facc6f2dcce16167a,,,,,\n\nfeline leukaemia virus (FeLV) p27 antigen ...,Morphologic features and development of granul...,[],"{'BIBREF0': {'ref_id': 'b0', 'title': 'Morphol..."
1,1d8907508d6c233f454408abbfd624a9fda81f37,,Yeon-Sook Kim,Yeon-Sook Kim (Chungnam National University Sc...,,Introduction\n\nMiddle East respiratory syndro...,Isolation of a novel coronavirus from a man wi...,"[{'first': 'Yeon-Sook', 'middle': [], 'last': ...","{'BIBREF0': {'ref_id': 'b0', 'title': 'Isolati..."
2,e0187a691be1237e922515d56bcfbc7ec988fa3e,Glucose 6-phosphate dehydrogenase deficiency e...,"H-C Yang, T-L Chen, Y-H Wu, K-P Cheng, Y-H Lin...","H-C Yang (Chang Gung University, Tao-Yuan, Tai...",Abstract\n\nGlucose 6-phosphate dehydrogenase ...,\n\nsusceptible to hydrogen peroxide (H 2 O 2 ...,Dietary regulation of expression of glucose-6-...,"[{'first': 'H-C', 'middle': [], 'last': 'Yang'...","{'BIBREF0': {'ref_id': 'b0', 'title': 'Dietary..."
3,da70e84ba93bb87634a681d7f61a44be543e8549,Rhinovirus Associated Severe Respiratory Failu...,"Kiwook Kim, Yeon Han Song, Joo-Hyun Park, Hye ...","Kiwook Kim, Yeon Han Song, Joo-Hyun Park, Hye ...",Abstract\n\nRhinovirus infection is typically ...,Introduction\n\nRhinovirus infection is typica...,Virological studies in natural common colds in...,"[{'first': 'Kiwook', 'middle': [], 'last': 'Ki...","{'BIBREF0': {'ref_id': 'b0', 'title': 'Virolog..."
4,98227f184bc6a0b098b4ae6387f97264aaf13654,EF-G catalyzed translocation dynamics in the p...,"Hee-Kyung Kim, Ignacio Tinoco","Hee-Kyung Kim (University of California, 94720...",Abstract\n\nProgrammed -1 ribosomal frameshift...,INTRODUCTION\n\nRibosomal frameshifting is pro...,"Programmed translational frameshifting, P J Fa...","[{'first': 'Hee-Kyung', 'middle': [], 'last': ...","{'BIBREF0': {'ref_id': 'b0', 'title': 'Program..."
